In [2]:
import pandas as pd
import seaborn
from nltk.corpus import stopwords
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer,HashingVectorizer
from sklearn.svm import SVC
import spacy
import string
from spacy.lang.en import English
from nltk.stem.porter import PorterStemmer
spacy.load('en')
parser = English()

In [3]:
STOPWORDS = set(stopwords.words('english') + list(ENGLISH_STOP_WORDS)) 
SYMBOLCHARS = " ".join(string.punctuation).split(" ") + ["-", "...", "”", "”","''"]

In [4]:
OffensiveLangDF = pd.read_csv('../Data/Offensive Language Dataset/Cleaned_labeled_data.csv')
spamSmsDF = pd.read_csv('../Data/SMS Spam Dataset/Cleaned_SMSSpamCollection.csv')
politicalDF = pd.read_csv('../Data/Indian Political Tweets Dataset/cleaned-tweets.csv')

currentDF = politicalDF

In [5]:
def tokenizeText(textData):

    textData = textData.strip().replace("\n", " ").replace("\r", " ")
    textData = textData.lower()
    tokens = parser(textData)

    lemmas = []
    for tok in tokens:
        lemmas.append(tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_)
    tokens = lemmas
    
    # Remove Stop Words
    tokens = [tok for tok in tokens if tok.lower() not in STOPWORDS]
    # Remove Symbols
    tokens = [tok for tok in tokens if tok not in SYMBOLCHARS]
    # Remove words with less than 3 characters
    tokens = [tok for tok in tokens if len(tok) >= 3]
    # Remove Non-Alphabetic Characters
    tokens = [tok for tok in tokens if tok.isalpha()]
    
    # Stemming of Words
    porter = PorterStemmer()
    tokens = [porter.stem(word) for word in tokens]
    
    tokens = list(set(tokens))
    textData = ' '.join(tokens[:])
    return textData

In [6]:
# hateSpeechDF['text'] = hateSpeechDF['text'].apply(lambda x:tokenizeText(x))
currentDF['text'] = currentDF['text'].apply(lambda x:tokenizeText(x))

In [7]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(currentDF['text'], currentDF['category'])
print (train_x)

5697      like queen treat india view nation lang special
2717    lost pakistan caus ghirti hui india support di...
3119                                          time go tdp
2067    defeat insiderupa polit minu elect india relig...
1857        work anybodi fake want diddi let reason think
                              ...                        
1285                     select costum final angel fallen
4286    effect bihar chief bureaucraci india rejig nit...
1181                     love great rate kid right gorgou
2442    cemetari visit haifa narendramodi live india i...
5815    prime rememb india leader shri rajivgandhi vis...
Name: text, Length: 4545, dtype: object


## Feature Extraction

### Count Vector as Features

In [8]:
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(currentDF['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [9]:
xtrain_count.shape

(4545, 10204)

### Word Level TF-IDF

In [10]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(currentDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

In [11]:
xtrain_tfidf.shape

(4545, 5000)

### N-Gram TF-IDF

In [12]:
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(currentDF['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

In [13]:
xtrain_tfidf.shape

(4545, 5000)

### Character Level TF-IDF

In [14]:
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(currentDF['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [15]:
xtrain_tfidf.shape

(4545, 5000)

## Model

In [16]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)

    return metrics.classification_report(predictions, valid_y)

## Training and Classification Reports

### SVM | Count

In [17]:
accuracy = train_model(SVC(kernel='linear',decision_function_shape='ovo'), xtrain_count, train_y, xvalid_count)
print(accuracy)

              precision    recall  f1-score   support

      NOTPOL       0.95      0.88      0.92       568
         POL       0.93      0.97      0.95       947

    accuracy                           0.94      1515
   macro avg       0.94      0.93      0.93      1515
weighted avg       0.94      0.94      0.94      1515



### SVM | Word level TF-IDF

In [18]:
accuracy = train_model(SVC(kernel='linear',decision_function_shape='ovo'), xtrain_tfidf, train_y, xvalid_tfidf)
print(accuracy)

              precision    recall  f1-score   support

      NOTPOL       0.98      0.88      0.92       584
         POL       0.93      0.99      0.96       931

    accuracy                           0.94      1515
   macro avg       0.95      0.93      0.94      1515
weighted avg       0.95      0.94      0.94      1515



### SVM | N-Gram level TF-IDF

In [19]:
accuracy = train_model(SVC(kernel='linear',decision_function_shape='ovo'), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print(accuracy)

              precision    recall  f1-score   support

      NOTPOL       0.14      0.92      0.25        83
         POL       0.99      0.69      0.81      1432

    accuracy                           0.70      1515
   macro avg       0.57      0.80      0.53      1515
weighted avg       0.95      0.70      0.78      1515



### SVM | Character level TF-IDF

In [20]:
accuracy = train_model(SVC(kernel='linear',decision_function_shape='ovo'), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print(accuracy)

              precision    recall  f1-score   support

      NOTPOL       0.95      0.86      0.90       580
         POL       0.92      0.97      0.94       935

    accuracy                           0.93      1515
   macro avg       0.93      0.92      0.92      1515
weighted avg       0.93      0.93      0.93      1515

